![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.Clinical_DeIdentificiation.ipynb)

# Clinical Deidentification

## Colab Setup

In [1]:
import json

with open('spark_nlp_for_healthcare_summit.json') as f:
    license_keys = json.load(f)

license_keys.keys()


dict_keys(['SECRET', 'SPARK_NLP_LICENSE', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'JSL_VERSION', 'PUBLIC_VERSION', 'SPARK_OCR_LICENSE', 'OCR_VERSION', 'JSL_OCR_SECRET'])

In [2]:
license_keys['JSL_VERSION']

'2.6.2'

In [3]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

secret = license_keys['SECRET']

os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
jsl_version = license_keys['JSL_VERSION']
version = license_keys['PUBLIC_VERSION']

! pip install --ignore-installed -q pyspark==2.4.4

! python -m pip install --upgrade spark-nlp-jsl==$jsl_version  --extra-index-url https://pypi.johnsnowlabs.com/$secret

! pip install --ignore-installed -q spark-nlp==$version

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

spark = sparknlp_jsl.start(secret)

In [ ]:
# if you want to start the session with custom params as in start function above
def start(secret):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:"+version) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+secret+"/spark-nlp-jsl-"+jsl_version+".jar")
      
    return builder.getOrCreate()

#spark = start(secret)

In [ ]:
spark

# Deidentification Model

Protected Health Information: 
- individual’s past, present, or future physical or mental health or condition
- provision of health care to the individual
- past, present, or future payment for the health care 

Protected health information includes many common identifiers (e.g., name, address, birth date, Social Security Number) when they can be associated with the health information.

Load NER pipeline to isentify protected entities:

In [4]:
import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel
import pyspark.sql.functions as F
import string
import numpy as np
import sparknlp
from sparknlp.util import *
from sparknlp.pretrained import ResourceDownloader
from pyspark.sql import functions as F
from sparknlp_jsl.annotator import *

In [154]:
from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on n2c2 (de-identification and Heart Disease Risk Factors Challenge) datasets)

clinical_ner = NerDLModel.pretrained("ner_deid_large", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverterInternal()\
  .setInputCols(["sentence", "token", "ner"])\
  .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_large download started this may take some time.
Approximate size to download 13.9 MB
[OK!]


### Pretrained NER models extracts:

- Name
- Profession
- Age
- Date
- Contact(Telephone numbers, FAX numbers, Email addresses)
- Location (Address, City, Postal code, Hospital Name, Employment information)
- Id (Social Security numbers, Medical record numbers, Internet protocol addresses)

In [155]:
text ='''
A . Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR . # 7194334 Date : 01/13/93 PCP : Oliveira , 25 month years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street
'''

In [156]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [157]:
result_df = result.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label"))

In [158]:
result_df.select("token", "ner_label").groupBy('ner_label').count().orderBy('count', ascending=False).show(truncate=False)

+----------+-----+
|ner_label |count|
+----------+-----+
|O         |28   |
|I-LOCATION|5    |
|I-NAME    |3    |
|B-NAME    |3    |
|B-DATE    |3    |
|B-LOCATION|2    |
|B-ID      |1    |
|B-AGE     |1    |
+----------+-----+



### Check extracted sensetive entities

In [159]:
result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------------+---------+
|chunk                        |ner_label|
+-----------------------------+---------+
|2093-01-13                   |DATE     |
|David Hale                   |NAME     |
|Hendrickson , Ora            |NAME     |
|7194334                      |ID       |
|01/13/93                     |DATE     |
|Oliveira                     |NAME     |
|25                           |AGE      |
|2079-11-09                   |DATE     |
|Cocke County Baptist Hospital|LOCATION |
|0295 Keats Street            |LOCATION |
+-----------------------------+---------+



We can find the cases, where the model will skip some important entities, for example:

In [160]:
text ='''
Patient AIQING, 25 years-old , born in Beijing, was transfered to the The Johns Hopkins Hospital. Phone number: (541) 754-3010. MSW 100009632582
'''

In [161]:
result2 = model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [163]:
result_df = result2.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label"))

result2.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+--------------------------+---------+
|chunk                     |ner_label|
+--------------------------+---------+
|25                        |AGE      |
|Beijing                   |LOCATION |
|The Johns Hopkins Hospital|LOCATION |
|(541) 754-3010            |CONTACT  |
|100009632582              |ID       |
+--------------------------+---------+



For these entities we can add a dictionary to the pipeline, by using **NerOverwriter()**:

In [164]:
neroverwriter = NerOverwriter() \
    .setInputCols(["ner"]) \
    .setOutputCol("ner_overwrited") \
    .setStopWords(['AIQING']) \
    .setNewResult("I-NAME")

ner_converter = NerConverterInternal()\
  .setInputCols(["sentence", "token", "ner_overwrited"])\
  .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    neroverwriter,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

Let's test the model after modification:

In [166]:
result2 = model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [167]:
result_df2 = result.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label"))

result_df2.show()

+----------+----------+
|     token| ner_label|
+----------+----------+
|   Patient|         O|
|    AIQING|         O|
|         ,|         O|
|        25|     B-AGE|
| years-old|         O|
|         ,|         O|
|      born|         O|
|        in|         O|
|   Beijing|B-LOCATION|
|         ,|         O|
|       was|         O|
|transfered|         O|
|        to|         O|
|       the|         O|
|       The|B-LOCATION|
|     Johns|I-LOCATION|
|   Hopkins|I-LOCATION|
|  Hospital|I-LOCATION|
|         .|         O|
|     Phone|         O|
+----------+----------+
only showing top 20 rows



In [168]:
result2.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+--------------------------+---------+
|chunk                     |ner_label|
+--------------------------+---------+
|AIQING                    |NAME     |
|25                        |AGE      |
|Beijing                   |LOCATION |
|The Johns Hopkins Hospital|LOCATION |
|(541) 754-3010            |CONTACT  |
|100009632582              |ID       |
+--------------------------+---------+



As we can see, now name **AIQING** was identified correctly

### Excluding entities from deidentification

Sometimes we need to leave some entities in the text, for example, if we want to analyze the frequency of the disease by the hospital. In this case, we need to use parameter **setWhiteList()** to modify NerChunk output. This parameter having using a list of entities type to deidentify as an input. So, if we want to leave the location in the list we need to remove this tag from the list:

In [170]:
ner_converter = NerConverterInternal()\
  .setInputCols(["sentence", "token", "ner_overwrited"])\
  .setOutputCol("ner_chunk") \
  .setWhiteList(['NAME', 'PROFESSION', 'ID', 'AGE',
               'DATE', 'CONTACT'])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    neroverwriter,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model_with_white_list = nlpPipeline.fit(empty_data)

In [171]:
text ='''
A . Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR . # 7194334 Date : 01/13/93 PCP : Oliveira , 25 month years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street
'''

result_with_white_list = model_with_white_list.transform(spark.createDataFrame([[text]]).toDF("text"))

In [172]:
result_df = result.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label"))

result_with_white_list.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------+---------+
|chunk            |ner_label|
+-----------------+---------+
|2093-01-13       |DATE     |
|David Hale       |NAME     |
|Hendrickson , Ora|NAME     |
|7194334          |ID       |
|01/13/93         |DATE     |
|Oliveira         |NAME     |
|25               |AGE      |
|2079-11-09       |DATE     |
+-----------------+---------+



## Masking and Obfuscation

### Replace this enitites with Tags

In [173]:
deidentification = DeIdentificationModel.pretrained("deidentify_large", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("mask")

deidentify_large download started this may take some time.
Approximate size to download 188.1 KB
[OK!]


In [178]:
text ='''
A . Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR . # 7194334 Date : 01/13/93 PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street
'''
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [179]:
result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------------+---------+
|chunk                        |ner_label|
+-----------------------------+---------+
|2093-01-13                   |DATE     |
|David Hale                   |NAME     |
|Hendrickson , Ora            |NAME     |
|7194334                      |ID       |
|01/13/93                     |DATE     |
|Oliveira                     |NAME     |
|25                           |AGE      |
|2079-11-09                   |DATE     |
|Cocke County Baptist Hospital|LOCATION |
|0295 Keats Street            |LOCATION |
+-----------------------------+---------+



In [180]:
deid_text = deidentification.transform(result)

In [181]:
deid_text.select(F.explode(F.arrays_zip('sentence.result', 'deidentified.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,A .,A .
1,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , <NAME> , M.D ."
2,", Name : Hendrickson , Ora MR .",", Name : <NAME> MR ."
3,"# 7194334 Date : 01/13/93 PCP : Oliveira , 25 years-old , Record date : 2079-11-09 .","# <ID> Date : <DATE> PCP : <NAME> , <AGE> years-old , Record date : <DATE> ."
4,Cocke County Baptist Hospital .,<LOCATION> .
5,0295 Keats Street,<LOCATION>


### Use obfuscation mode

In the obfuscation mode **DeIdentificationModel** will replace sensetive entities with random values of the same type. 

Will be replaced: 
- Name
- Location
- Contacts
- Profession

Will be tagged:
- Age
- Date
- ID

In [182]:
obfuscation = DeIdentificationModel.pretrained("deidentify_large", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("obfuscate")\
      .setObfuscateDate(True)

        #.setObfuscateRefFile('obfuscation.txt') \
        #.setRefSep('#') \
        #.setRegexPatternsDictionary('regex_dict.txt', 'TEXT')
      

deidentify_large download started this may take some time.
Approximate size to download 188.1 KB
[OK!]


sample obfuscation.txt
```
Marvin MARSHALL#DOCTOR
Hubert GROGAN#DOCTOR
ALTHEA COLBURN#DOCTOR
Kalil AMIN#DOCTOR
Inci FOUNTAIN#DOCTOR
Ekaterina Rosa#DOCTOR
Rudiger Chao#DOCTOR
COLLETTE KOHLER#DOCTOR
Mufi HIGGS#DOCTOR
```

sample regex_dict.txt
```
AGE \d{1,2}\s(?:months|weeks|mos)
MEDICALRECORD (?<=MRN\s)(\w+
```

In [183]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|      ner_overwrited|           ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|
A . Record date ...|[[document, 0, 21...|[[document, 1, 3,...|[[token, 1, 1, A,...|[[word_embeddings...|[[named_entity, 1...|[[named_entity, 1...|[[chunk, 19, 28, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [184]:
obfusated_text = obfuscation.transform(result)

In [185]:
obfusated_text.select('ner.result').take(1)

[Row(result=['O', 'O', 'O', 'O', 'O', 'B-DATE', 'O', 'B-NAME', 'I-NAME', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NAME', 'I-NAME', 'I-NAME', 'O', 'O', 'O', 'B-ID', 'O', 'O', 'B-DATE', 'O', 'O', 'B-NAME', 'O', 'B-AGE', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION'])]

In [186]:
obfusated_text.select(F.explode(F.arrays_zip('sentence.result', 'deidentified.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,A .,A .
1,"Record date : 2093-01-13 , David Hale , M.D .","Record date : 2093-02-23 , Jere , M.D ."
2,", Name : Hendrickson , Ora MR .",", Name : IGNACIUS MR ."
3,"# 7194334 Date : 01/13/93 PCP : Oliveira , 25 years-old , Record date : 2079-11-09 .","# <ID> Date : <DATE> PCP : Jephthah , <AGE> years-old , Record date : 2079-11-19 ."
4,Cocke County Baptist Hospital .,Irena .
5,0295 Keats Street,Graford


## Use full pipeline in the Light model

In [187]:
finisher = Finisher() \
    .setInputCols("deidentified")

ner_converter = NerConverterInternal()\
  .setInputCols(["sentence", "token", "ner_overwrited"])\
  .setOutputCol("ner_chunk") 

In [188]:
pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    neroverwriter,
    ner_converter,
    obfuscation])

In [189]:

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

In [190]:
light_model = LightPipeline(model)

In [191]:
text ='''
A . Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR . # 7194334 Date : 01-13-1993 PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street
'''

In [192]:
annotated_text = light_model.annotate(text)
annotated_text['deidentified']

['A .',
 'Record date : 2093-03-13 , Jere , M.D .',
 ', Name : IGNACIUS MR .',
 '# <ID> Date : 01-26-1993 PCP : Jephthah , <AGE> years-old , Record date : 2079-11-22 .',
 'Irena .',
 'Graford']

In [193]:
list(zip(annotated_text['token'], annotated_text['ner']))

[('A', 'O'),
 ('.', 'O'),
 ('Record', 'O'),
 ('date', 'O'),
 (':', 'O'),
 ('2093-01-13', 'B-DATE'),
 (',', 'O'),
 ('David', 'B-NAME'),
 ('Hale', 'I-NAME'),
 (',', 'O'),
 ('M.D', 'O'),
 ('.', 'O'),
 (',', 'O'),
 ('Name', 'O'),
 (':', 'O'),
 ('Hendrickson', 'B-NAME'),
 (',', 'I-NAME'),
 ('Ora', 'I-NAME'),
 ('MR', 'O'),
 ('.', 'O'),
 ('#', 'O'),
 ('7194334', 'B-ID'),
 ('Date', 'O'),
 (':', 'O'),
 ('01-13-1993', 'B-DATE'),
 ('PCP', 'O'),
 (':', 'O'),
 ('Oliveira', 'B-NAME'),
 (',', 'O'),
 ('25', 'B-AGE'),
 ('years-old', 'O'),
 (',', 'O'),
 ('Record', 'O'),
 ('date', 'O'),
 (':', 'O'),
 ('2079-11-09', 'B-DATE'),
 ('.', 'O'),
 ('Cocke', 'B-LOCATION'),
 ('County', 'I-LOCATION'),
 ('Baptist', 'I-LOCATION'),
 ('Hospital', 'I-LOCATION'),
 ('.', 'O'),
 ('0295', 'B-LOCATION'),
 ('Keats', 'I-LOCATION'),
 ('Street', 'I-LOCATION')]

In [194]:
annotated_text['ner_chunk']

['2093-01-13',
 'David Hale',
 'Hendrickson , Ora',
 '7194334',
 '01-13-1993',
 'Oliveira',
 '25',
 '2079-11-09',
 'Cocke County Baptist Hospital',
 '0295 Keats Street']

In [195]:
source_text = '''Record date : 2093-01-13, David Hale, M.D. is manager, 
Name: Hendrickson, Ora MR. # 7194334 Date: 01-13-1993 PCP: Oliveira.
Record date: 2079-11-09. Cocke County Baptist Hospital. 0295 Keats Street.
This 17-yr-old male, presented with chest heaviness that started during a pick-up basketball game. His past medical history was unremarkable. He denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age 38. The patient was a nonsmoker, did not drink alcohol, and denied recreational drug use. He swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal.'''

annotated_text = light_model.annotate(source_text)
annotated_text['deidentified']

['Record date : 2093-03-02, Jere, M.D. is Costume Attendants, \nName: IGNACIUS MR. # <ID> Date: (859)247-1693 PCP: Jephthah.',
 'Record date: 2080-01-07.',
 'Irena.',
 'Graford.',
 'This <AGE> male, presented with chest heaviness that started during a pick-up basketball game.',
 'His past medical history was unremarkable.',
 'He denied prior cardiac symptoms and suffered no chest trauma during the game.',
 'His father had suffered an acute myocardial infarction at age',
 '<AGE>. The patient was a nonsmoker, did not drink alcohol, and denied recreational drug use.',
 'He swallowed a tablet of aspirin before coming to the emergency room.',
 'His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm.',
 'Physical examination revealed no stigmata of Marfan syndrome.',
 'The rest of his physical examination was normal.']

## Train custom Model


In [65]:
glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

public_ner = NerDLModel.pretrained("ner_dl", 'en') \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

ner_converter = NerConverterInternal()\
  .setInputCols(["sentence", "token", "ner"])\
  .setOutputCol("ner_chunk")


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]


In [66]:

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    glove_embeddings,
    public_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

custom_model = nlpPipeline.fit(result)

In [139]:
source_text = '''Record date : 2093-01-13, David Hale lives in New York'''

In [140]:
result = custom_model.transform(spark.createDataFrame([[source_text]]).toDF("text"))

#### We got new entities in the model 'PER' and 'ORG'

In [141]:
result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+----------+---------+
|chunk     |ner_label|
+----------+---------+
|David Hale|PER      |
|New York  |LOC      |
+----------+---------+



### Mask custom entities

In [142]:
deidentification = DeIdentification() \
      .setInputCols(["sentence", "token", "ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("mask")

In [143]:
deid_model = deidentification.fit(result)
deid_text = deid_model.transform(result)

In [144]:
deid_text.select(F.explode(F.arrays_zip('sentence.result', 'deidentified.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,"Record date : 2093-01-13, David Hale lives in New York","Record date : 2093-01-13, <PER> lives in <LOC>"


In [145]:
deid_model.write().overwrite().save('custom_deid_masker_model')

 To be able to obfuscate new entities we have to create custom Deidentification Model with new dictionary

In [146]:
obfuscation_list = ['Marvin MARSHALL#PER',
      'Hubert GROGAN#PER',
      'ALTHEA COLBURN#PER',
      'Kalil AMIN#PER',
      'Inci FOUNTAIN#PER',
      'Surrey#ORG',
      'Warwickshire#ORG',
      'Derbyshire#ORG',
      'Leicestershire#ORG',
      'Glamorgan#ORG',
      'Durham#ORG',
      'Los Angeles#LOC']

with open('obfuscation.txt', 'a') as the_file:
    for line in obfuscation_list:
      the_file.write(line + '\n')

In [147]:
obfuscator = DeIdentification() \
    .setInputCols(["sentence", "token", "ner_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode('obfuscate') \
    .setRefSep('#') \
    .setObfuscateRefFile('obfuscation.txt') \
    .setObfuscateDate(True)


In [148]:
obfuscator_model = obfuscator.fit(result)

In [149]:
obfuscator_model.write().overwrite().save('custom_obfuscator_model')

In [150]:
obfusc_text = obfuscator_model.transform(result)

In [151]:
obfusc_text.select(F.explode(F.arrays_zip('sentence.result', 'deidentified.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,"Record date : 2093-01-13, David Hale lives in New York","Record date : 2093-01-13, Kalil AMIN lives in New York"
